<a href="https://colab.research.google.com/github/ehdghks316/itwill_studyGroup_D/blob/main/4_19%ED%8C%8C%EC%9D%B4%EC%8D%AC(overfitting%2Cunderfitting%2Ck_fold%2C_clustering%2C_k_means).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
★ overfitting
- 학습데이터(training set)에 대해 과하게 학습된 상황을 의미한다.
- 학습데이터에 이외의 데이터(test set)에 대해서는 모델의 정확도가 낮다.
- 데이터 모델의 특성에 비해 모델이 너무 복잡한 경우
- 학습데이터가 부족할 경우

★ underfitting
- 학습데이터부터 정확도가 낮다.
- 데이터 양이 너무 적을 경우 발생
- 반복횟수가 너무 적을 경우 발생
- 데이터 모델의 특성에 비해 모델이 너무 간단한 경우

★ k-fold 교차검증
- 기존 데이터를 k개로 나눠서 k번 정확도를 검증하는 방법
- 예를 들어 5-fold 면 학습데이터를 5조각으로 나눠 1조각을 검증데이터로 사용하고 나머지 4조각을 학습데이터 사용한다.
- 첫번째 조각부터 5번째 조각까지 한번씩 검증하게 되어 결과적으로 5개의 검증결과에 해당하는 정확도를 평균을 검증결과의 점수로 표현한다.
- 레이블(정답)의 분포도가 좋을 때 사용한다.

★ stratified k fold
분균형한 분포도를 가진 레이블(정답) 데이터셋을 위한 방식


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from collections import Counter

iris = pd.read_csv('c:/data/iris.csv')
iris.info()
iris.head()
iris.iloc[:,0:4]
iris.Name

x_train, x_test, y_train, y_test = train_test_split(iris.iloc[:,0:4],iris.Name,test_size=0.1,random_state=123)
x_train.shape
y_train.shape
Counter(y_train)
Counter(y_test)

knn_clf = KNeighborsClassifier()
parameters = {'n_neighbors' : [3,5,7,9,11]} # 파라미터값은 딕셔너리형식으로 값은 1개라도 무조건 리스트로

grid_knn = GridSearchCV(estimator=knn_clf, param_grid=parameters,cv=5,refit=True,return_train_score=True)
grid_knn.fit(x_train,y_train)
pd.DataFrame(grid_knn.cv_results_)[['params','mean_test_score','rank_test_score']]

grid_knn.best_params_ # 최적의 파라미터 값
grid_knn.best_score_ # 최고 정확도

y_pred = grid_knn.predict(x_test)
accuracy_score(y_test,y_pred)

knn_estimator = grid_knn.best_estimator_ # GridSearchVC의 refit으로 이미 학습된 estimator를 반환 / 만든 최적의 모델(사용될)
y_pred = knn_estimator.predict(x_test)
accuracy_score(y_test,y_pred)


★ clustering(군집화)
- unsupervised learning
- 유사한 성질을 가지는 데이터끼리 cluster(군집)를 나누는 과정
- 동일한 집단에 소속된 관측치들은 서로 유사할수록 좋다.
- 상이한 집단에 소속된 관측치들은 서로 다를수록 좋다.
- 군집내 데이터들의 거리는 가깝고 군집단 거리는 멀게 하는게 군집이 잘 되어 있다.
- 유사한 뉴스그룹끼리 묶어 놓은 것
- 특허문서분석
- 관련된 문서들을 모을 때

# 유사한 문서들끼리 뽑아내보기( knn )
# 라벨없이 군집을 나누고 라벨을 넣었을 때 정확도?

★ k-means
- 주어진 데이터에서 k개의 cluster로 묶는 알고리즘

k-mean 동장방식
1. 랜덤으로 중심지정
2. 각중심으로부터 가까운 데이터들에 레벨을 붙인다.
3. 각 레벨의 중심(평균)으로 이동
4. 클러스터의 중심이 더 이상 변하지 않으면 중단

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
iris = pd.read_csv('c:/data/iris.csv')

model = KMeans(n_clusters=3)
model.fit(iris.iloc[:,0:-1]) # 학습
model.labels_ # 학습된 라벨 추출
model.cluster_centers_ # 중심값들 추출

iris['cluster'] = model.labels_
iris

colormap = np.array(['red','blue','black'])
plt.scatter(iris.PetalLength,iris.PetalWidth,c=colormap[model.labels_],s=40)

model.inertia_ 
inertia 값은 군집화된 후에 각 중심점에서 군집의 데이터간의 거리를 합산한 것이므로 군집의 응집도를 나타내는 값
inertia 값이 작을수록 응집도가 높게 군집화가 잘되어 있다고 평가할 수 있다.

# k값 찾는 과정
inertia = []
for k in range(1,10):
    model = KMeans(n_clusters=k)
    model.fit(iris.iloc[:,0:4])
    inertia.append(model.inertia_)

plt.plot(range(1,10),inertia,'-o')
# 꺾이는 지점 : 엘보우 지점에서 k값을 찾는게 좋다(1번 빼고)



#-------------------------------------------------------------------------------------------
k-means 프로젝트 : snsdata로 1.운동종목 군집화, 2.나이 빈칸에 들어갈 값(평균), 3.젠더 전처리?(기존 데이터들을 분석해서 성별에 넣기)
snsdata = pd.read_csv('c:/data/snsdata.csv')

# 젠더 null값 제외하기
snsdata = snsdata[~snsdata['gender'].isnull()]


# 나이 전처리 -> nan값이 아닌 나이의 평균값을 nan값에 넣기
snsdata['age'][snsdata['age'].isnull()] = snsdata['age'][~snsdata['age'].isnull()].mean()
sum(snsdata['age'].isnull()) # null값이 있는지 확인

snsdata['age'][snsdata['age'].rank(ascending=False) < 60]
snsdata['age'][snsdata['age'].rank(ascending=True) < 60]

# friend로 군집화
model = KMeans(n_clusters=3)
model.fit(snsdata.iloc[:,3:4])
model.labels_
model.inertia_
model.cluster_centers_
inertia = []
for k in range(1,20):
    model = KMeans(n_clusters=k)
    model.fit(snsdata.iloc[:,3:4])
    inertia.append(model.inertia_)
    
plt.plot(range(1,20),inertia,'-o')

plt.scatter(snsdata.iloc[:,3:4],snsdata.iloc[:,3:4],c=colormap[model.labels_],s=20)

# 나이, friend로 군집화
snsdata.iloc[:,2:4]
model = KMeans(n_clusters=3)
model.fit(snsdata.iloc[:,2:4])
model.labels_
model.inertia_
model.cluster_centers_
plt.scatter(snsdata.age,snsdata.friends,c=colormap[model.labels_],s=20)

# 운동종목 군집화
model = KMeans(n_clusters=9)
model.fit(snsdata.iloc[:,4:13])
model.labels_
model.inertia_
model.cluster_centers_

colormap = np.array(['black','blue','red','yellow','orange','purple','green','grey','brown'])
plt.scatter(snsdata.basketball,snsdata.football,c=colormap[model.labels_],s=20)


